In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

c:\Users\Dmitry\Miniconda3\envs\AI\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
image_size = (256, 256)
batch_size = 16

In [11]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    # x = data_augmentation(inputs)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # for size in [128, 256, 512, 728]:
    for size in [256, 512]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=image_size + (3,), num_classes=2)
keras.utils.plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [12]:
image_size = (256, 256)
batch_size = 16

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "made_datasets\hypothesis1_256",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    shuffle=True,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "made_datasets\hypothesis1_256",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    shuffle=True
)


Found 2231 files belonging to 2 classes.
Using 1785 files for training.
Found 2231 files belonging to 2 classes.
Using 446 files for validation.


In [8]:
class_weight = {0: 1.,
                1: 50.}

In [9]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [10]:
epochs = 5

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=METRICS,
)

model.fit(
    train_ds, epochs=epochs, validation_data=val_ds
)

Epoch 1/5
112/112 [==============================] - 212s 2s/step - loss: 0.7399 - tp: 461.0000 - fp: 323.0000 - tn: 573.0000 - fn: 428.0000 - accuracy: 0.5793 - precision: 0.5880 - recall: 0.5186 - auc: 0.6147 - prc: 0.5862 - val_loss: 0.6923 - val_tp: 148.0000 - val_fp: 120.0000 - val_tn: 99.0000 - val_fn: 79.0000 - val_accuracy: 0.5538 - val_precision: 0.5522 - val_recall: 0.6520 - val_auc: 0.5702 - val_prc: 0.6095
Epoch 2/5
112/112 [==============================] - 188s 2s/step - loss: 0.6628 - tp: 524.0000 - fp: 289.0000 - tn: 607.0000 - fn: 365.0000 - accuracy: 0.6336 - precision: 0.6445 - recall: 0.5894 - auc: 0.6839 - prc: 0.6745 - val_loss: 0.6868 - val_tp: 227.0000 - val_fp: 219.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.5090 - val_precision: 0.5090 - val_recall: 1.0000 - val_auc: 0.6313 - val_prc: 0.6976
Epoch 3/5
112/112 [==============================] - 194s 2s/step - loss: 0.6323 - tp: 560.0000 - fp: 300.0000 - tn: 596.0000 - fn: 329.0000 - accurac